A feature of MotionClouds is the ability to precisely tune the precision of information  following the principal axes. One which is particularly relevant for the primary visual cortical area of primates (area V1) is to tune the otirentation mean and bandwidth.

## Studying the role of contrast in V1 using MotionClouds

<!-- TEASER_END -->

This is part of a larger study to tune [orientation bandwidth](https://laurentperrinet.github.io/sciblog/categories/orientation.html).



In [1]:
import os
import numpy as np
import MotionClouds as mc
downscale = 1
fx, fy, ft = mc.get_grids(mc.N_X//downscale, mc.N_Y//downscale, mc.N_frame//downscale)
name = '2016-04-06_IRM-protocol'
mc.figpath = os.path.join('../files/', name)
if not(os.path.isdir(mc.figpath)): os.mkdir(mc.figpath)

In [2]:
N_X = fx.shape[0]
width = 29.7*256/1050
sf_0 = 4.*width/N_X
B_V = .5     # BW temporal frequency (speed plane thickness)
B_sf = sf_0   # BW spatial frequency
theta = 0.0   # Central orientation
B_theta_low, B_theta_high = np.pi/12, 2*np.pi 
seed = 12234565

mc1 = mc.envelope_gabor(fx, fy, ft, V_X=0., V_Y=0., B_V=B_V, sf_0=sf_0, B_sf=B_sf, theta=theta, B_theta=B_theta_low)
mc2 = mc.envelope_gabor(fx, fy, ft, V_X=0., V_Y=0., B_V=B_V, sf_0=sf_0, B_sf=B_sf, theta=theta, B_theta=B_theta_high)
name_ = name + '_narrow'
mc.figures(mc1, name_, seed=seed, figpath=mc.figpath)
mc.in_show_video(name_, figpath=mc.figpath)
name_ = name + '_broad'
mc.figures(mc2, name_, seed=seed, figpath=mc.figpath)
mc.in_show_video(name_, figpath=mc.figpath)

/usr/local/lib/python3.7/site-packages/vispy/visuals/isocurve.py:22: UserWarning: VisPy is not yet compatible with matplotlib 2.2+
  warnings.warn("VisPy is not yet compatible with matplotlib 2.2+")


This figure shows how one can create different MotionCloud stimuli that specifically target different population in V1. We show in the two lines of this table  motion cloud component with a (Top) narrow orientation bandwith  (Bottom) a wide bandwitdh: perceptually, there is no predominant position or speed, just different orientation contents.
<br>Columns represent isometric projections of a cube. The left column displays iso-surfaces of the spectral envelope by displaying enclosing volumes at 5 different energy values with respect to the peak amplitude of the Fourier spectrum.
<br>The middle column shows an isometric view of the  faces of the movie cube. The first frame of the movie lies on the x-y plane, the x-t plane lies on the top face and motion direction is seen as diagonal lines on this face (vertical motion is similarly see in the y-t face). The third column displays the actual movie as an animation.

Note: an analysis of the response of a neural network to such stimulations as been explored by [Chloé Pasturel](https://laurentperrinet.github.io/sciblog/posts/2014-06-18-stage-m1-chloe-pasturel-week-6.html).

## designing one block with different orientation

In [3]:
import os
def make_one_block(seed, B_theta, N_sub, B_V=0.5, N_frame_sub=6, N_theta=12):
    np.random.seed(seed)
    N_X, N_Y, N_frame = fx.shape
    im = np.zeros((N_X, N_Y, 0))
    disk = mc.frequency_radius(fx, fy, ft) < .5
    for i_sub in range(N_sub):
        theta = np.int(np.random.rand()*N_theta) * np.pi / N_theta
        mc_i = mc.envelope_gabor(fx, fy, ft, 
                                         V_X=0., V_Y=0., B_V=B_V, 
                                         sf_0=sf_0, B_sf=B_sf, 
                                         theta=theta, B_theta=B_theta)
        im = np.concatenate((im, (mc.rectif(mc.random_cloud(mc_i))*disk + .5*(1-disk))[:, :, :N_frame_sub]), axis=-1)
    return im
                
im = make_one_block(seed=1234, B_theta=mc.B_theta, N_sub=20, N_frame_sub=6, N_theta=12)
name_ = name + '_block'
mc.anim_save(im, os.path.join(mc.figpath, name_), figpath=mc.figpath)
mc.in_show_video(name_, figpath=mc.figpath)

""


Here, we display MotionClouds as the orientation bandwidth increases from pi/32 to 2*pi. This should be equivalent in V1 to recruiting different ratios of neurons. 
<br>Left column displays iso-surfaces of the spectral envelope by displaying enclosing volumes at 5 different energy values with respect to the peak amplitude of the Fourier spectrum. Right column of the table displays the actual movie as an animation.

## summary of the protocol

We show successive blocks of 2 seconds consisting of 20 sub-blocks of 100ms.
- only the orientation is changed within one block
- contrast and bandwidths are changed across blocks

To summarize this (POST = TO BE DONE AT THE PRESENTATION SOFTWARE LEVEL):

* Rapid presentation of 20 stimuli within a circular disk (POST) during 100 ms @ 60Hz = 6 frames each
* fixed parameters:
 - mean spatial frequency tuned for optimal neural tuning,
 - frequency bandwidth tuned for optimal neural tuning (0.1 - 5 cyc/deg),
 - temporal frequency bandwidth tuned for optimal neural tuning (1-15Hz (singh et al + Henriksson et al)) / one bandwidth in speed: dynamic (B_V=.5) or static (my preference for this short period)

* parameters:
 - 12 orientations including cardinals
 - 7 orientation bandwidths (infinite, pi/2, pi/4, pi/8, pi/16, pi/32, dirac),
 - 3 different seeds: 42, 1973 and 2015 (completely arbitrary)
 - 4 different contrasts 0.03 0.07 0.18 0.42 (Boynton et al) (POST)

Grand total for one block is (12 orientations times 6 BWo + 1) * 2s :

In [4]:
print('One super-block=', (7*3), ' conditions')
print('One super-block=', (7*3) * 2, ' seconds')
print('16 repetitions of one super-block=', (7*3) * 16, ' conditions')
print('16 repetitions of one super-block=', (7*3) * 2 * 16, ' seconds')

One super-block= 21  conditions
One super-block= 42  seconds
16 repetitions of one super-block= 336  conditions
16 repetitions of one super-block= 672  seconds


One session amounts  $672$ seconds, that is about $10$ minutes. 

Let's first get the optimal values for the
 - mean spatial frequency tuned for optimal neural tuning,
 - spatial frequency bandwidth tuned for optimal neural tuning

In [5]:
viewingDistance = 57 # cm # TODO = me donner ces informations!
screen_width_cm = 33 # cm # TODO = me donner ces informations!
#un deg / cm
print('visual angle of the screen', 2*np.arctan(screen_width_cm/2/viewingDistance)*180/np.pi)
print('degrees per centimeter', 2*np.arctan(screen_width_cm/2/viewingDistance)*180/np.pi/screen_width_cm)

visual angle of the screen 32.28867756056697
degrees per centimeter 0.9784447745626353


In [6]:
screen_width_px = 1024 # pixels 
screen_height_px = 768 # pixels

#un pixel = 33/800 deg
deg_per_px = 2*np.arctan(screen_width_cm/2/viewingDistance)*180/np.pi/screen_width_px
print('degrees per pixel', deg_per_px)

degrees per pixel 0.03153191168024118


The central spatial frequency ``sf_0`` is defined as the frequency (number of cycles) *per pixel*, so that to get 

In [7]:
print('width of these motion clouds (', mc.N_X, ', ', mc.N_Y, ')')
print('width of stimulus in degrees', mc.N_X * deg_per_px)
phi_sf_0 = 2. # Optimal spatial frequency [cpd]
print('Optimal spatial frequency in cycles per degree', phi_sf_0)
print('Optimal spatial frequency in cycles per window = ', phi_sf_0 *  mc.N_X * deg_per_px)
sf_0 = phi_sf_0 * deg_per_px
print('cycles per pixel = ', sf_0)

width of these motion clouds ( 256 ,  256 )
width of stimulus in degrees 8.072169390141742
Optimal spatial frequency in cycles per degree 2.0
Optimal spatial frequency in cycles per window =  16.144338780283483
cycles per pixel =  0.06306382336048236


Similarly the spatial frequeny bandwidth as a function of the experimental parameters:

In [8]:
phi_sf_0 = 2. # Optimal spatial frequency [cpd]
phi_B_sf = 2. # Optimal spatial frequency bandwidth [in octaves]
B_Sf = sf_0 # good qualitative approximation

In [9]:
phi_B_V = 5. # Optimal temporal frequency bandwidth [Hz]

#tf_opt = 1 # Hz
T = 0.250            # Stimulus duration [s] 
framerate = 100.    # Refreshing rate in [Hz]
Bv = phi_B_V # good qualitative approximation 

In one script:

In [10]:
%%writefile ../files/2016-04-06_IRM-protocol/2016-04-06_IRM-protocol.py
import numpy as np
import MotionClouds as mc
import os

#downscale = 1
#fx, fy, ft = mc.get_grids(mc.N_X//downscale, mc.N_Y//downscale, mc.N_frame//downscale)
fx, fy, ft = mc.get_grids(512, 512, 128)

name = '2016-04-06_IRM-protocol'
vext = '.png'
mc.figpath = '.'
if not(os.path.isdir(mc.figpath)): os.mkdir(mc.figpath)

# Experimental constants 
contrast = 1.
# Clouds parameters in absolute units
N_X = fx.shape[0]
width = 29.7*256/1050
phi_sf_0 = 2. # Optimal spatial frequency [cpd]

sf_0 = phi_sf_0*width/N_X
B_sf = sf_0   # BW spatial frequency
B_V = .5     # BW temporal frequency (speed plane thickness) WARNING temporal autocorrelation depends on N_frame

# generate zip files
dry_run = True
dry_run = False

def make_one_block(seed, B_theta, N_sub, B_V=B_V, N_frame_sub=6, N_theta=12):
    np.random.seed(seed)
    N_X, N_Y, N_frame = fx.shape
    im = np.zeros((N_X, N_Y, 0))
    disk = mc.frequency_radius(fx, fy, ft) < .5

    for i_sub in range(N_sub):
        theta = np.int(np.random.rand()*N_theta) * np.pi / N_theta
        mc_i = mc.envelope_gabor(fx, fy, ft, 
                                         V_X=0., V_Y=0., B_V=B_V, 
                                         sf_0=sf_0, B_sf=B_sf, 
                                         theta=theta, B_theta=B_theta)
        im = np.concatenate((im, (mc.rectif(mc.random_cloud(mc_i))*disk + .5*(1-disk))[:, :, :N_frame_sub]), axis=-1)
    return im
                
#for i_B_theta, B_theta in enumerate([np.pi/24, np.pi/12, np.pi/6, np.pi/3, np.pi][::-1]):
for i_B_theta, B_theta in enumerate([np.pi/12, np.inf][::-1]):
    if B_theta == np.inf: B_theta_str = 'inf'
    else: B_theta_str = str(int(np.ceil(B_theta*180/np.pi)))
    for seed in [2016 + i for i in range(7)]:
        name_ = name + '_B_theta_' + B_theta_str
        name_ += '_seed_' + str(seed)
        if not dry_run:
            if  not(os.path.isfile(os.path.join(mc.figpath, name, name_ + vext))):
                im = make_one_block(seed=seed+i_B_theta, B_theta=B_theta, N_sub=20, N_frame_sub=6, N_theta=12)
                mc.anim_save(mc.rectif(im, contrast=contrast), os.path.join(mc.figpath, name, name_), vext=vext)
            else:
                print(' MC ' + os.path.join(mc.figpath, name, name_) + ' already done')
        else:
            print(' MC ' + os.path.join(mc.figpath, name, name_) + ' skipped  (dry run)')


Overwriting ../files/2016-04-06_IRM-protocol/2016-04-06_IRM-protocol.py


In [11]:
%%writefile  ../files/2016-04-06_IRM-protocol/2016-04-06_IRM-protocol.sh
cd ../files/2016-04-06_IRM-protocol/
rm -fr 2016-04-06_IRM-protocol.zip 2016-04-06_IRM-protocol
ipython3 2016-04-06_IRM-protocol.py
zip 2016-04-06_IRM-protocol.zip 2016-04-06_IRM-protocol 2016-04-06_IRM-protocol/* 2016-04-06_IRM-protocol/**/*
rm -fr 2016-04-06_IRM-protocol
echo 'done'

Overwriting ../files/2016-04-06_IRM-protocol/2016-04-06_IRM-protocol.sh


In [12]:
!sh ../files/2016-04-06_IRM-protocol/2016-04-06_IRM-protocol.sh

]0;IPython: files/2016-04-06_IRM-protocol  adding: 2016-04-06_IRM-protocol/ (stored 0%)
  adding: 2016-04-06_IRM-protocol/2016-04-06_IRM-protocol_B_theta_15_seed_2016/ (stored 0%)
  adding: 2016-04-06_IRM-protocol/2016-04-06_IRM-protocol_B_theta_15_seed_2017/ (stored 0%)
  adding: 2016-04-06_IRM-protocol/2016-04-06_IRM-protocol_B_theta_15_seed_2018/ (stored 0%)
  adding: 2016-04-06_IRM-protocol/2016-04-06_IRM-protocol_B_theta_15_seed_2019/ (stored 0%)
  adding: 2016-04-06_IRM-protocol/2016-04-06_IRM-protocol_B_theta_15_seed_2020/ (stored 0%)
  adding: 2016-04-06_IRM-protocol/2016-04-06_IRM-protocol_B_theta_15_seed_2021/ (stored 0%)
  adding: 2016-04-06_IRM-protocol/2016-04-06_IRM-protocol_B_theta_15_seed_2022/ (stored 0%)
  adding: 2016-04-06_IRM-protocol/2016-04-06_IRM-protocol_B_theta_inf_seed_2016/ (stored 0%)
  adding: 2016-04-06_IRM-protocol/2016-04-06_IRM-protocol_B_theta_inf_seed_2017/ (stored 0%)
  adding: 2016-04-06_IRM-protocol/2016-04-06_IRM-protocol_B_theta_inf_seed_2018/

## some book keeping for the notebook

In [13]:
%load_ext version_information
%version_information numpy, scipy, matplotlib, MotionClouds

Software versions
Python 3.7.1 64bit [Clang 10.0.0 (clang-1000.11.45.5)]
IPython 7.1.1
OS Darwin 17.7.0 x86_64 i386 64bit
numpy 1.15.4
scipy 1.1.0
matplotlib 3.0.1
MotionClouds 20180606
Wed Nov 14 15:47:56 2018 CET